In [1]:
import pandas as pd
import os
import torch 
import clip
import numpy as np
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device,"device")

/users/eleves-b/2022/hanna.mergui/Computer-Vision/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda device


In [2]:
summary_bert_path = "/users/eleves-b/2022/hanna.mergui/Computer-Vision/ComputerVision_Data/Summaries/export_summary_Bert.csv"
summary_bert = pd.read_csv(summary_bert_path)

data_dir = "ComputerVision_Data"
images_dir = "Food Images/Food Images"

In [3]:
recipe = summary_bert["Summary"]
print("number of sumary recipe ",len(recipe))

images = os.listdir(os.path.join(data_dir, images_dir))
print("number of images ",len(images))

number of sumary recipe  13463
number of images  3486


In [4]:
import torch

# Get the current selected GPU
device = torch.cuda.current_device()

# Get the name of the current selected GPU
print(torch.cuda.get_device_name(device))

# Get the available memory in bytes
print(torch.cuda.memory_allocated(device))
print(torch.cuda.memory_reserved(device))

NVIDIA RTX A2000 12GB
0
0


In [5]:
models = clip.available_models()
print(models)
model, preprocess = clip.load('RN50', device,jit=False)

['RN50', 'ViT-B/32']


In [6]:

#Tensor creation and array creation


context_length=77
array_text=[]

# Creation of a tensor for the text 
device = "cuda" if torch.cuda.is_available() else "cpu"
#print(device,"device")
text_tensor = torch.zeros(len(summary_bert), context_length, dtype=torch.long)
labels = []
for i, row in enumerate(summary_bert.iterrows()):
    summary = row[1]["Summary"]
    if len(summary)>context_length:
        summary = summary[:context_length]
    array_text.append(summary)
    text_tensor[i] = clip.tokenize(summary, context_length).to(device)  # Access row data using integer indices
    labels.append(row[1]["Image_Name"])  # Access row data using integer indices

torch.save(text_tensor, "text_tensor.pt")
    
print(array_text[0])

Roast chicken in a large cast-iron skillet until an instant-read thermometer 


In [7]:
text_tensor = torch.load("text_tensor.pt")
print(text_tensor.shape)
print(len(array_text))

torch.Size([13463, 77])
13463


In [8]:
images_inputs=[]

for im in images:
    im_path = os.path.join(data_dir, images_dir, im)
    images_inputs.append(preprocess(Image.open(im_path)))
images_inputs_tensor = torch.tensor(np.stack(images_inputs)).to(device)

In [12]:
print("text_tensor shape", text_tensor.shape)
print("images_inputs_tensor shape ", images_inputs_tensor.shape)

print("images array shape",len(images_inputs))

text_tensor shape torch.Size([13463, 77])
images_inputs_tensor shape  torch.Size([3486, 3, 224, 224])
images array shape 3486


In [10]:
# Doenst work

"""
test_image = images_inputs[0].unsqueeze(0).to(device)
scores=[]  # Use the context length from the text tensor
array_text=[]
context_length=150

print("test_image shape ", test_image.shape)

for i, row in enumerate(summary_bert.iterrows()):
        summary = row[1]["Summary"]
        text = clip.tokenize(summary, context_length).to(device)  # Convert tensor type to LongTensor


        print("test text shape ", text.shape)
        with torch.no_grad():
                
                image_features = model.encode_image(test_image)
                #print("image feature ", image_features.shape)
                
                #text_features = text_features.unsqueeze(0)  # Reshape text_features to match image_features
                text_features = model.encode_text(text)
                
                #print("text_features",text_features.shape)

                logits_per_image, logits_per_text = model(test_image, text)  # Use text_features instead of text
                probs = logits_per_image.softmax(dim=-1).to(device).cpu().numpy()
        
        print("Label probs:", probs) 
    
"""



'\ntest_image = images_inputs[0].unsqueeze(0).to(device)\nscores=[]  # Use the context length from the text tensor\narray_text=[]\ncontext_length=150\n\nprint("test_image shape ", test_image.shape)\n\nfor i, row in enumerate(summary_bert.iterrows()):\n        summary = row[1]["Summary"]\n        text = clip.tokenize(summary, context_length).to(device)  # Convert tensor type to LongTensor\n\n\n        print("test text shape ", text.shape)\n        with torch.no_grad():\n                \n                image_features = model.encode_image(test_image)\n                #print("image feature ", image_features.shape)\n                \n                #text_features = text_features.unsqueeze(0)  # Reshape text_features to match image_features\n                text_features = model.encode_text(text)\n                \n                #print("text_features",text_features.shape)\n\n                logits_per_image, logits_per_text = model(test_image, text)  # Use text_features instead of text\

In [14]:
image = preprocess(Image.open("ComputerVision_Data/Food Images/Food Images/-bloody-mary-tomato-toast-with-celery-and-horseradish-56389813.jpg")).unsqueeze(0).to(device)
#text = clip.tokenize(["a diagram", "a dog", "a cat"]).to(device)

#text = clip.tokenize(text_tensor, context_length).to(device)
text_list = text_tensor.tolist()
print(text_list)

text = clip.tokenize(["a diagram", "a dog", "a cat"], context_length).to(device)

print("image",image.shape)
print("text",text.shape)

with torch.no_grad():
    #print("image is",image)
    #print("text is",text)
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]
print("best prob",max(probs[0]))


[[49406, 11404, 3717, 530, 320, 3638, 1970, 268, 5391, 40568, 1942, 550, 10635, 268, 1199, 38172, 49407, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [49406, 11404, 518, 11567, 1942, 1070, 16458, 537, 9838, 827, 628, 1243, 593, 320, 8960, 267, 272, 276, 339, 49407, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [49406, 2430, 33352, 943, 5205, 537, 2954, 5205, 531, 320, 14318, 564, 7508, 530, 320, 3638, 2096, 2632, 550, 15456, 49407, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [49406, 5066, 753, 655, 23533, 274, 1161, 3158, 537, 7737, 263, 2654, 2077, 264, 530, 320, 13358, 3365, 536, 31227, 49407, 0, 0, 0, 0